In [3]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from datetime import datetime

# === 1. Cargar el modelo entrenado ===
with open("modelo_lightgbm_final.pkl", "rb") as f:
    model = pickle.load(f)

# === 2. Cargar la nueva data de predicción ===
df_pred = pd.read_parquet("data_202507.parquet")
df_pred.columns = df_pred.columns.str.lower()

# Procesamiento mínimo como antes
df_pred.fillna(0, inplace=True)
df_pred["codzona_x"] = df_pred["codzona_x"].replace("", -1).astype(int)
df_pred["codregion"] = df_pred["codregion"].replace("", -1).astype(int)
df_pred = df_pred.astype({
    "codterritorio": int,
    "codzona_x": int,
    "codregion": int,
    "aniocampanaingreso": int,
    "campanaingreso": int,
    "aniocampana": int,
})

# === 3. Preparar X_test ===
X_test = df_pred.drop(["codebelista", "clase", "aniocampana"], axis=1)
X_test = X_test.apply(pd.to_numeric, errors='coerce').fillna(0)

# === 4. Predecir probabilidades ===
probs = model.predict(X_test)
df_pred["probs"] = probs

# === 5. Clasificar en deciles y riesgo ===
df_pred["proba_decil"] = pd.qcut(
    df_pred["probs"].rank(method="first"),
    10,
    labels=False
)

df_pred["Perfil_Riesgo"] = np.where(
    df_pred.proba_decil < 5, "1 - Riesgo Bajo",
    np.where(df_pred.proba_decil < 8, "2 - Riesgo Medio", "3 - Riesgo Alto")
)

# === 6. Guardar resultados ===
fecha_str = datetime.now().strftime("%Y%m%d_%H%M%S")
result_cols = ["aniocampana", "codebelista", "codregion", "probs", "Perfil_Riesgo"]
output_path = f"predicciones_resultado_{fecha_str}.csv"

df_pred[result_cols].to_csv(output_path, index=False)
print(f"✅ Resultados guardados en {output_path}")


AttributeError: 'super' object has no attribute 'get_params'

In [3]:
df_pred.head()

,aniocampanaproceso,aniocampana,codebelista,codterritorio,codcomportamientorolling,edad,flagcorreovalidado,flagcelular,flagdigital,desestadocivil_cat,...,flag19,flag20,desv_pedmin,desv_pedmin_adj,clase,target_cobranza31_ca,prom_dias_pago,probs,proba_decil,Perfil_Riesgo
0,202507,202507,052359198,37017,5,35,0,1,0,5,...,0,0,160.94000,177.83430,1,1,28.15385,0,0,1 - Riesgo Bajo
1,202507,202507,052359260,8092,7,38,0,1,0,5,...,0,0,225.69000,223.23440,1,1,144.77778,0,0,1 - Riesgo Bajo
2,202507,202507,052311764,165818,7,33,0,1,0,5,...,0,0,57.08000,60.59450,1,1,24.72727,0,0,1 - Riesgo Bajo
3,202507,202507,035822933,19112,5,56,0,1,0,5,...,0,0,244.24000,227.20000,1,1,27.91667,0,0,1 - Riesgo Bajo
4,202507,202507,052315611,34818,7,23,0,1,0,5,...,0,0,143.49000,175.20150,1,1,108.50000,0,0,1 - Riesgo Bajo
